In [144]:
import numpy as np
import pandas as pd
import requests as rq
from sklearn.pipeline import Pipeline

In [107]:
weather_types = { 'NA': 'Not available', '-1': 'Trace rain', '0': 'Clear night', '1': 'Sunny day', '2': 'Partly cloudy (night)', '3': 'Partly cloudy (day)', '4': 'Not used', '5': 'Mist', '6': 'Fog', '7': 'Cloudy', '8': 'Overcast', '9': 'Light rain shower (night)', '10': 'Light rain shower (day)', '11': 'Drizzle', '12': 'Light rain', '13': 'Heavy rain shower (night)', '14': 'Heavy rain shower (day)', '15': 'Heavy rain', '16': 'Sleet shower (night)', '17': 'Sleet shower (day)', '18': 'Sleet', '19': 'Hail shower (night)', '20': 'Hail shower (day)', '21': 'Hail', '22': 'Light snow shower (night)', '23': 'Light snow shower (day)', '24': 'Light snow', '25': 'Heavy snow shower (night)', '26': 'Heavy snow shower (day)', '27': 'Heavy snow', '28': 'Thunder shower (night)', '29': 'Thunder shower (day)', '30': 'Thunder'}

visibility_dict = { 'UN': 'Unknown', 'VP': 'Very poor - Less than 1 km', 'PO': 'Poor - Between 1-4 km', 'MO': 'Moderate - Between 4-10 km', 'GO': 'Good - Between 10-20 km', 'VG': 'Very good - Between 20-40 km', 'EX': 'Excellent - More than 40 km'}
 

In [104]:
def fetch_api_data():
    data_url = "http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/3772?res=hourly&key=12a3f254-6c4a-4f6d-a464-a7a2290a7226"
    response = rq.get(data_url)
    return response.json()


In [150]:
def get_location(data):
    return f"{data['SiteRep']['DV']['Location']['name']}, {data['SiteRep']['DV']['Location']['country']}"

'HEATHROW, ENGLAND'

In [151]:
def make_dataframe(data):
    weather_data = None
    for day in data['SiteRep']['DV']['Location']['Period']:
        df = pd.DataFrame(day['Rep'])
        df['date'] = day['value']
        weather_data = pd.concat([weather_data, df], ignore_index=True)
    return weather_data

In [115]:
def dataframe_manipulations(weather_data):
    headers = {'D':'Wind Direction(compass)', 'G':'Wind Gust(mph)', 'H':'Screen Relative Humidity(%)', 'P':'Pressure(hpa)', 'S':'Wind Speed(mph)', 'T':'Temperature(C)', 'V':'Visibility(m)', 'W':'Weather Type', 'Pt':'Pressure Tendency', 'Dp':'Dew Point(C)', '$':'Minutes Since 12o Clock'}
    weather_data.rename(columns=headers, inplace=True)
    compass_directions_map = { 'N': 1, 'NNE':2, 'NE': 3, 'ENE':4, 'E': 5, 'ESE':6, 'SE': 7, 'SSE':8, 'S': 9, 'SSW':10, 'SW': 11, 'WSW':12, 'W': 13, 'WNW':14, 'NW': 15, 'NNW':16}
    weather_data['Wind Direction(compass)'] = weather_data['Wind Direction(compass)'].map(compass_directions_map)
    Pressure_tendency_map = { 'F': 0, 'R': 1, 'S': 2}   
    weather_data['Pressure Tendency'] = weather_data['Pressure Tendency'].map(Pressure_tendency_map)
    weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y-%m-%dZ')
    column_datatypes = {'Wind Direction(compass)':int, 'Wind Gust(mph)':float, 'Screen Relative Humidity(%)':float, 'Pressure(hpa)':float, 'Wind Speed(mph)':float, 'Temperature(C)':float, 'Visibility(m)':float, 'Weather Type':int, 'Pressure Tendency':int, 'Dew Point(C)':float, 'Minutes Since 12o Clock':int}
    weather_data = weather_data.astype(column_datatypes)
    weather_data['day'] = weather_data['date'].dt.day
    weather_data['month'] = weather_data['date'].dt.month
    weather_data['year'] = weather_data['date'].dt.year
    weather_data.drop('date', axis=1, inplace=True)
    return weather_data

In [ ]:
# use this to streamline the process
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Import your functions
from your_module import fetch_data_using_api, apply_custom_transformations, create_dataframe, train_model

# Wrap your functions with FunctionTransformer
fetch_transformer = FunctionTransformer(fetch_data_using_api, validate=False)
transform_transformer = FunctionTransformer(apply_custom_transformations, validate=False)
df_transformer = FunctionTransformer(create_dataframe, validate=False)
train_transformer = FunctionTransformer(train_model, validate=False)

# Create a pipeline
pipeline = Pipeline([
    ('fetch', fetch_transformer),             # Fetch data using API
    ('transform', transform_transformer),     # Apply custom transformations
    ('create_df', df_transformer),            # Create DataFrame
    ('train', train_transformer)              # Train model
])

# Now, you can use the pipeline to process new data
new_data = ...  # Your new data in the required format

# Fit and transform the new data through the pipeline
pipeline.fit_transform(new_data)


In [ ]:
import torch
import torch.nn as nn

class MultiInputMultiOutputNN(nn.Module):
    def __init__(self, input1_dim, input2_dim, output1_dim, output2_dim, hidden_dim):
        super(MultiInputMultiOutputNN, self).__init__()

        # Define separate input layers for each input
        self.input1_layer = nn.Sequential(
            nn.Linear(input1_dim, hidden_dim),
            nn.ReLU()
        )
        self.input2_layer = nn.Sequential(
            nn.Linear(input2_dim, hidden_dim),
            nn.ReLU()
        )

        # Shared hidden layers
        self.shared_layer = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU()
        )

        # Output layers for each output
        self.output1_layer = nn.Linear(hidden_dim, output1_dim)
        self.output2_layer = nn.Linear(hidden_dim, output2_dim)

    def forward(self, input1, input2):
        # Process each input separately
        input1_output = self.input1_layer(input1)
        input2_output = self.input2_layer(input2)

        # Combine the processed inputs
        combined_output = torch.cat((input1_output, input2_output), dim=1)

        # Pass through shared hidden layer
        shared_output = self.shared_layer(combined_output)

        # Separate output predictions
        output1 = self.output1_layer(shared_output)
        output2 = self.output2_layer(shared_output)

        return output1, output2

# Define the input and output dimensions
input1_dim = 10
input2_dim = 5
output1_dim = 2
output2_dim = 3
hidden_dim = 20

# Create an instance of the model
model = MultiInputMultiOutputNN(input1_dim, input2_dim, output1_dim, output2_dim, hidden_dim)

# Example input data
input1_data = torch.randn(32, input1_dim)
input2_data = torch.randn(32, input2_dim)

# Forward pass through the model
output1, output2 = model(input1_data, input2_data)
print("Output 1:", output1.shape)
print("Output 2:", output2.shape)
